In [ ]:
# specify commonly used column names
column_name_all = df_source.columns
column_name_basic, column_name_smart = column_name_all[:5], column_name_all[5:]
column_name_smart_raw = pd.Index([c for c in column_name_smart if "raw" in c ])
column_name_smart_normalized = pd.Index([c for c in column_name_smart if "normalized" in c ])
print(column_name_basic)
print(column_name_smart_raw)
print(column_name_smart_normalized)

In [ ]:
# setup for reading time series
#serials = ['ZLW0EGC6', 'Z305B2QN', 'ZLW0C6NE', 'ZJV0XJQ3', 'ZLW18MKT', 'ZLW0GGTP']
serials = ['ZLW0EGC6', 'ZLW0GGTP']
start_date = "2021-10-01"
end_date = "2021-10-31"
days = pd.Period(end_date).dayofyear - pd.Period(start_date).dayofyear + 1
column_to_save = ['date', 'serial_number', 'model', 'capacity_bytes', 'failure', 'smart_1_normalized']
#column_to_save = ['date', 'capacity_bytes', 'failure', 'smart_1_normalized']
#column_to_save = column_name_all

In [ ]:
# create a list of empty dataframes for concatenation later
time_series_dataset = []
for serial in serials:
    time_series_dataset.append(pd.DataFrame([], columns=column_to_save))

memory_used = 0
memory_warning_displayed = False
print(f"Read files for building time series from {start_date} to {end_date}; totally {days} days; {len(serials)} serials")
for day in range(days):
    date = pd.Period(start_date) + day
    print(f"  reading data for the date {date}; progress: {day+1}/{days}")
    file_path = "../data/data_Q4_2021/" + str(date) + ".csv"
    df_tmp = pd.read_csv(file_path, parse_dates=["date"])
    for i, serial in enumerate(serials):
        df_tmp_serial = df_tmp.query("serial_number == @serial")[column_to_save]
        time_series_dataset[i] = pd.concat([time_series_dataset[i], df_tmp_serial], ignore_index=True)
        memory_used += time_series_dataset[i].memory_usage(deep=True).sum()
    if memory_used/1024**3 > 1 and not memory_warning_displayed:
        print(" ### Warning: memory used for time-series dataframe > 1 GB ###")
        memory_warning_displayed = True

print("Time series was read successfully")
print(f"Memory used for dataframe: {(memory_used/1024**3).round(3)} GB")

In [ ]:
time_series_dataset[0].head()

**Save it as CSV**


In [ ]:
print("Save read time series to csv files")
for i, serial in enumerate(serials):
    file_path = "../data/time_series/" + serial + "_" + start_date + "_to_" + end_date + ".csv" 
    time_series_dataset[i].to_csv(file_path)
    print(f"  save to \"{file_path}\"; progress {i}/{len(serials)}")
print("Time series are saved successfully")